In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from resnet1d import Resnet34
# from resnet_zoo import Resnet34
from dataset import Dataset_ori
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torch.autograd import Variable
import os
import numpy as np
from tqdm import tqdm
import argparse
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc
import sys
import os

from collections import OrderedDict

In [2]:
def test_epoch(PPG_model, test_loader, baseline=False):
    with torch.no_grad():
       

        PPG_preds = None
        PPG_pred_probs = None
        all_targets = None
        
        PPG_model.eval()
        tstart = datetime.now()

        for batch_idx, (PPG, target) in enumerate(test_loader):

            PPG = PPG.cuda().float()
            target = target.cuda().long()
            
            _, PPG_out = model(PPG)

            PPG_predicted = PPG_out.argmax(1)
            PPG_predicted_prob = F.softmax(PPG_out, dim=1)[:, 1]

            if PPG_preds == None:
                PPG_pred_probs = PPG_predicted_prob
                PPG_preds = PPG_predicted
                all_targets = target
            else:
                PPG_preds = torch.cat((PPG_preds, PPG_predicted))
                PPG_pred_probs = torch.cat((PPG_pred_probs, PPG_predicted_prob))
                all_targets = torch.cat((all_targets, target))
        tend = datetime.now()

        precision, recall, thresholds = precision_recall_curve(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
        pr_auc = auc(recall, precision)

        print(f'[TEST] \tPPG      F1: {round(f1_score(all_targets.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG ROC AUC: {round(roc_auc_score(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG PR  AUC: {round(pr_auc, 4)}')

In [3]:
MODEL_PATH = '/home/users/zg78/adver_ml/baseline/saved_models/res34_epoch_30_ppglr_0.0001_/PPG_best_0.pt'
state_dict = torch.load(MODEL_PATH) 

In [4]:
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict

In [5]:
model = Resnet34().cuda()
model.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
data_folder = '/usr/xtmp/zg78/stanford_dataset/'
test_dataset = Dataset_ori(data_folder+'testx_accpt_clean.npy', data_folder+'testy_af_accpt_clean.npy')

testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)
test_epoch(model, testloader)


[TEST] 	PPG      F1: 0.8401
[TEST] 	PPG ROC AUC: 0.9931
[TEST] 	PPG PR  AUC: 0.9686
